In [1]:
import pandas as pd
import numpy as np
from environment import Environment
from pandas.tseries.offsets import MonthEnd
import instrument
from portfolio import Portfolio
from risk_parity import RiskParity



%load_ext autoreload
%autoreload 2

In [2]:
from historical import HistoricalData
from pandas._libs.tslibs.timestamps import Timestamp, Timedelta

path = "data\\"
env_df = pd.read_csv(path + "Consolidated.csv", header = [0,1], index_col=[0])
targets_df = pd.read_csv(path + "Targets.csv", index_col=[0])
features_df = pd.read_csv(path + "Features.csv", header = [0,1], index_col=[0])
exps = pd.read_csv('data\\ETF_info.csv', index_col=0).loc[env_df[['EQ', 'EM', 'RE', 'FI']].columns.droplevel(), 'Expense Ratio'].astype(float)
data = HistoricalData(env_df, features_df, targets_df)

date = Timestamp('2009-03-01') + MonthEnd(0)
date_end = date + MonthEnd(60)
env_now = Environment(*data.get_env_args(date))

rp = RiskParity(data)


### Initializing stuff

In [3]:
def dumb_optimize(env: Environment):
    w_dict = {}
    n_assets = len(env.prices.items())
    for asset, price in env.prices.items():
        w_dict[asset] = 1/n_assets
    return w_dict

usd = 200000/env_now.fx['CAD']
w_dict = rp.get_weights(date)
w_dict = dumb_optimize(env_now)
pos_dict = Portfolio.weights_to_pos(w_dict, env_now, usd)
pos_dict = Portfolio.etf_dict_from_names(pos_dict)
icash = instrument.Cash('USD')
pos_dict[icash] = 0

In [4]:
my_pf = Portfolio(pos_dict)
my_pf.rebalance(env_now, Portfolio.weights_to_pos(w_dict, env_now, my_pf.calc_value(env_now)*0.99), exps, time_past=0)

C = 0.97
nopt = my_pf.pf_units[my_pf.get_cash('USD')]
ttm=2

specs = [{'name':'SPY Put', 'ccy':'USD', 'is_call':0, 'ul':'SPY US Equity','K':my_pf.get_asset('SPY US Equity').value(env_now)*C}]

ofee = my_pf.buy_options(env_now, specs, [nopt], ttm=ttm, pos_array_type='Dollars')

### Set up performance df

In [5]:
perf_df = pd.DataFrame(index = pd.date_range(start=date, end=date_end, freq='M'))
perf_df['USD Value'] = 0
perf_df['CAD Value'] = 0
perf_df['Return'] = 0
perf_df['PNL'] = 0
perf_df['Injection'] = 0
perf_df['FX'] =  env_now.fx['CAD']
perf_df['Option Fees'] = ofee
perf_df['ETF Fees'] = 0
perf_df['ETF Expenses'] = 0

### Run sim/backtest

In [6]:
perf_df.loc[date, 'USD Value'] = my_pf.calc_value(env_now)
perf_df.loc[date, 'CAD Value'] = my_pf.calc_value(env_now, ccy='CAD')
perf_df.loc[date, 'Return'] = np.nan
perf_df.loc[date, 'PNL'] = np.nan

while date < date_end:

    date = date + MonthEnd(1)
    env_now = Environment(*data.get_env_args(date))
    #w_dict = rp.get_weights(date)
    perf_df.loc[date, 'USD Value'] = my_pf.calc_value(env_now)
    perf_df.loc[date, 'CAD Value'] = my_pf.calc_value(env_now, ccy='CAD')

    perf_df.loc[date, 'FX'] = env_now.fx['CAD']
    
    if (date.month == 8) or (date.month == 2):
        my_pf.pf_units[my_pf.get_cash('USD')] += 20000/env_now.fx['CAD']
        perf_df.loc[date, 'Injection'] = 20000/env_now.fx['CAD']
        
    perf_df.loc[date, 'Return'] = perf_df.loc[date, 'USD Value']/(perf_df.loc[date + MonthEnd(-1), 'USD Value'] + perf_df.loc[date + MonthEnd(-1), 'Injection']) - 1
    perf_df.loc[date, 'PNL'] = perf_df.loc[date, 'USD Value'] - (perf_df.loc[date + MonthEnd(-1), 'USD Value'] + perf_df.loc[date + MonthEnd(-1), 'Injection'])
    
    efee, eexps = my_pf.rebalance(env_now, Portfolio.weights_to_pos(w_dict, env_now, my_pf.calc_value(env_now)*0.99), exps)
    ofee = my_pf.sell_options(env_now)
    
    nopt = my_pf.pf_units[my_pf.get_cash('USD')]
    specs = [{'name':'SPY Put', 'ccy':'USD', 'is_call':0, 'ul':'SPY US Equity','K':my_pf.get_asset('SPY US Equity').value(env_now)*C}]
    ofee += my_pf.buy_options(env_now, specs, [nopt], ttm=ttm, pos_array_type='Dollars')
    perf_df.loc[date, 'ETF Fees'] = efee
    perf_df.loc[date, 'ETF Expenses'] = ofee

### Evaluate performance

In [7]:
perf_df

,USD Value,CAD Value,Return,PNL,Injection,FX,Option Fees,ETF Fees,ETF Expenses
2009-03-31,158558.168323,199815.003720,NaN,NaN,0.000000,1.26020,13.149143,0.000000,0.000000
2009-04-30,175437.262710,209208.935782,0.106454,16879.094387,0.000000,1.19250,13.149143,66.466402,27.503979
2009-05-31,184743.522189,201654.944210,0.053046,9306.259479,0.000000,1.09154,13.149143,54.450000,30.014995
2009-06-30,184000.566655,213873.058651,-0.004022,-742.955535,0.000000,1.16235,13.149143,70.158781,32.213184
2009-07-31,194298.334748,209377.828508,0.055966,10297.768093,0.000000,1.07761,13.149143,84.150000,33.239603
2009-08-31,199932.255024,218681.901900,0.028996,5633.920276,18285.212748,1.09378,13.149143,19.800000,33.982493
2009-09-30,225487.581294,241158.968194,0.033316,7270.113522,0.000000,1.06950,13.149143,74.250000,35.285304
2009-10-31,220691.004552,239394.567188,-0.021272,-4796.576742,0.000000,1.08475,13.149143,64.350000,34.818228
2009-11-30,229564.030024,242488.484914,0.040206,8873.025471,0.000000,1.05630,13.149143,79.200000,35.212124
2009-12-31,230286.719128,242549.486921,0.003148,722.689104,0.000000,1.05325,13.149143,79.200000,38.668341


In [8]:
irr_values = np.array([-perf_df['USD Value'].iloc[0]] + (-perf_df['Injection'].values).tolist())
irr_values[-1] += perf_df['USD Value'].iloc[-1]
np.irr(irr_values)*12

0.0854869267882874

In [9]:
m = perf_df['Return'].mean()*12
s = perf_df['Return'].std()*np.sqrt(12)
var = -np.percentile(perf_df['Return'].dropna(), 5)*np.sqrt(12)
cvar = -perf_df['Return'][perf_df['Return']<np.percentile(perf_df['Return'].dropna(), 5)].mean()*np.sqrt(12)
print("Mean: {:.1f}%\nStd: {:.1f}%\nVaR: {:.1f}%\nCVaR: {:.1f}%".format(m*100, s*100, var*100, cvar*100))
print("Sharpe: {:.1f}".format(m/s))

Mean: 10.6%
Std: 9.8%
VaR: 10.0%
CVaR: 12.9%
Sharpe: 1.1


In [10]:
w_dict

{'SPY US Equity': 0.037037037037037035,
 'EFA US Equity': 0.037037037037037035,
 'XLF US Equity': 0.037037037037037035,
 'XLK US Equity': 0.037037037037037035,
 'XLV US Equity': 0.037037037037037035,
 'XLP US Equity': 0.037037037037037035,
 'XLE US Equity': 0.037037037037037035,
 'EWJ US Equity': 0.037037037037037035,
 'EWZ US Equity': 0.037037037037037035,
 'XLU US Equity': 0.037037037037037035,
 'XLI US Equity': 0.037037037037037035,
 'EZU US Equity': 0.037037037037037035,
 'IYR US Equity': 0.037037037037037035,
 'XLB US Equity': 0.037037037037037035,
 'RWR US Equity': 0.037037037037037035,
 'IXN US Equity': 0.037037037037037035,
 'ISMUF US Equity': 0.037037037037037035,
 'ICF US Equity': 0.037037037037037035,
 'IYZ US Equity': 0.037037037037037035,
 'ILF US Equity': 0.037037037037037035,
 'IEV US Equity': 0.037037037037037035,
 'TIP US Equity': 0.037037037037037035,
 'AGG US Equity': 0.037037037037037035,
 'IEF US Equity': 0.037037037037037035,
 'TLT US Equity': 0.037037037037037035

In [11]:
targets_df

,SPY US Equity,EFA US Equity,XLF US Equity,XLK US Equity,XLV US Equity,XLP US Equity,XLE US Equity,EWJ US Equity,XLU US Equity,XLI US Equity,...,IV1M120,IV2M80,IV2M90,IV2M95,IV2M975,IV2M100,IV2M1025,IV2M105,IV2M110,IV2M120
2006-06-30,0.002250,-0.000612,-0.004599,-0.004895,0.000346,0.018381,0.025402,-0.025854,0.022959,-0.001279,...,-0.015480,-0.125923,-0.113340,-0.105975,-0.108805,-0.110007,-0.109952,-0.102541,-0.049894,0.063655
2006-07-31,0.004873,0.008106,0.022880,-0.026071,0.054563,0.015710,0.033479,-0.005090,0.048003,-0.056786,...,-0.089623,0.027767,0.036297,0.039723,0.050812,0.060606,0.055906,0.049545,0.117318,0.063707
2006-08-31,0.021823,0.025485,0.011790,0.070200,0.028535,0.038664,-0.050810,0.010415,0.027482,0.011290,...,0.023316,-0.101018,-0.098985,-0.124318,-0.135662,-0.151128,-0.153715,-0.136802,-0.124000,-0.112523
2006-09-30,0.026993,0.002219,0.040231,0.040311,0.015350,-0.000494,-0.036606,-0.012477,-0.014537,0.038148,...,0.167932,0.006969,0.010141,0.010388,0.006216,0.008857,-0.001009,-0.024554,0.021689,0.036810
2006-10-31,0.031517,0.037490,0.023396,0.040454,0.003313,0.017732,0.042655,0.019960,0.052958,0.028196,...,-0.016618,-0.102941,-0.118795,-0.110350,-0.102703,-0.091308,-0.066667,-0.029748,-0.020112,-0.033531
2006-11-30,0.019885,0.030730,0.007056,0.027520,-0.001201,-0.008516,0.085772,0.000000,0.026544,0.024503,...,-0.211609,0.089682,0.056329,0.035439,0.022375,0.006763,-0.006494,-0.035377,-0.103763,-0.092857
2006-12-31,0.013369,0.031990,0.037614,-0.005392,0.010446,0.025070,-0.028153,0.036266,0.008553,0.004887,...,-0.009320,-0.014159,0.031156,0.034226,0.037037,0.032630,0.014161,-0.002445,-0.035623,0.088864
2007-01-31,0.015040,0.013931,0.009255,0.020207,0.030457,0.020291,-0.009211,0.007103,-0.003540,0.020851,...,0.206021,-0.071813,-0.061592,-0.063309,-0.069805,-0.080855,-0.092374,-0.101716,-0.058047,-0.010331
2007-02-28,-0.019617,-0.000809,-0.030473,-0.028655,-0.024053,-0.013130,-0.020485,0.031309,0.050834,-0.008954,...,0.067083,0.246615,0.237152,0.295699,0.328098,0.375126,0.434320,0.470668,0.337535,0.106472
2007-03-31,0.011602,0.028040,-0.004712,0.011713,0.002853,0.018349,0.062914,-0.012844,0.038115,0.005821,...,-0.103070,0.026765,0.017518,0.000000,-0.012484,-0.026471,-0.047855,-0.075139,-0.092147,-0.100000
